## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-21-49-50 +0000,wapo,Live updates: Israeli strike targeting Hamas i...,https://www.washingtonpost.com/world/2025/09/0...
1,2025-09-09-21-49-00 +0000,wsj,Opinion | Israel Targets Hamas in Qatar,https://www.wsj.com/opinion/israel-doha-qatar-...
2,2025-09-09-21-47-00 +0000,wsj,Suspect in North Carolina Stabbing Charged Wit...,https://www.wsj.com/us-news/suspect-in-north-c...
3,2025-09-09-21-47-00 +0000,wsj,Supreme Court Agrees to Fast-Track Trump’s Tar...,https://www.wsj.com/us-news/law/supreme-court-...
4,2025-09-09-21-46-00 +0000,wsj,Opinion | Another K-12 Education Disaster,https://www.wsj.com/opinion/naep-scores-readin...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,44
90,new,24
83,nepal,17
8,israel,16
200,protests,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
76,2025-09-09-20-09-00 +0000,nypost,Trump was not on board with Israel’s strike in...,https://nypost.com/2025/09/09/us-news/trump-wa...,115
256,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,97
129,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...,94
211,2025-09-09-12-48-55 +0000,bbc,White House denies Trump's alleged birthday me...,https://www.bbc.com/news/articles/cvgqnn4ngvdo...,84
92,2025-09-09-19-49-00 +0000,wsj,The Trump administration on Tuesday released i...,https://www.wsj.com/politics/policy/takeaways-...,83


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
76,115,2025-09-09-20-09-00 +0000,nypost,Trump was not on board with Israel’s strike in...,https://nypost.com/2025/09/09/us-news/trump-wa...
117,80,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...
276,78,2025-09-09-07-15-58 +0000,wapo,"At least 19 killed, hundreds injured in protes...",https://www.washingtonpost.com/world/2025/09/0...
96,64,2025-09-09-19-29-47 +0000,nypost,Sicko accused of brutally stabbing Ukrainian r...,https://nypost.com/2025/09/09/us-news/ukrainia...
336,47,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
279,46,2025-09-09-06-41-09 +0000,bbc,Thai court rules ex-PM Thaksin must serve one ...,https://www.bbc.com/news/articles/cly7k2g37g4o...
277,43,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
223,42,2025-09-09-11-48-49 +0000,bbc,Israeli military orders all Gaza City resident...,https://www.bbc.com/news/articles/cvg47kvld8go...
142,41,2025-09-09-17-05-42 +0000,nypost,"US test scores hit damning new lows in math, r...",https://nypost.com/2025/09/09/us-news/us-test-...
46,37,2025-09-09-20-56-56 +0000,nypost,"Singer D4vd cooperating with police, will cont...",https://nypost.com/2025/09/09/us-news/singer-d...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
